## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
# импортируем панду, без неё не обойтись
import pandas as pd

# читаем файл с входными данными для проекта
data = pd.read_csv('/datasets/data.csv')

# смотрим общую информацию по датасету
data.info()

# смотрим первые 15 строк датасета
data.head(15)

# здесь я максимально новичково (имеется ввиду малый опыт, а не синее бельё) удостоверилась, что строки,
# где отсутствуют значения заработка и опыта работы, совпадают
# days_employed_na_indices = data.index[data['days_employed'].isna()]
# total_income_na_indices = data.index[data['total_income'].isna()]
# compare = days_employed_na_indices == total_income_na_indices
# false_count = 0
# for element in compare:
#     if element == 'False':
#         false_count = false_count + 1
# false_count
# len(compare)
        
    
# здесь я ещё посмотрела, сколько уникальных количеств детей бывает, потом закомментила,
# чтобы не накладывалось на предыдущие команды
# data['children'].value_counts()
# data[data['children']==20]

# далее смотрю, много ли отрицательных значений для трудового стажа (закомменчено тоже)
# data[data['days_employed'] < 0].count()

# потом смотрю колонку возраста, какие там есть значения, и заодно, сколько их (закомменчено)
# data['dob_years'].value_counts()
# смотрю, что за личности, которым по 0 лет
# data[data['dob_years']==0].head(50)

# смотрю уникальные варианты в колонке про образование, потом закомменчиваю эту команду
# data['education'].unique()

# колонка про семейный статус, варианты и количества, закомменчено
# data.groupby('family_status').count()

# смотрю половое распределение заёмщиков и закомменчиваю это
# data['gender'].value_counts()

# смотрю виды занятости, а также убеждаюсь, что в колонке про задолженности всего два значения - 0 и 1
# data['income_type'].value_counts()
# data['debt'].value_counts()

# далее смотрю доходы, максимальный и минимальный, вдруг у кого отрицательный или космически огромный
# data['total_income'].max()
# data['total_income'].min()

# поищем дубликаты целых строк:
# data.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Так, вроде бы я вспомнила, как начать писать в Markdown. Надо просто два раза щёлкнуть на поле!

Ок. Я вижу, что в наших данных:

1. Разное количество элементов в столбцах. В столбцах со стажем и доходом есть пропуски. Причём, судя по одинаковому количеству ненулевых значений, скорее всего пропуски в одних и тех же местах (индексах). Что было бы логично, ведь нет стажа -> нет работы (официальной) -> нет дохода (официального). Проверили: так и есть. Итого: 2174 пропуска довольно важных для принятия решения о предоставлении кредита данных.  
    
2. Постолбцовые наблюдения: 
    2.1. У 76 человек по 20 детей!! Такая многодетность из разряда рекордов Гиннеса, либо это семьи, где берут много детей из приютов, но что-то тоже много. Или там лишние нолики. В пользу ошибки говорит то, что в данных отсутствуют люди с 6-19 и 21 и более детьми.
    В задаче требуется определить влияние количества детей, но в вопросах при этом говорится только о наличии. Если изучать только наличие, то эти "многодетности" нам никак не помешают. Но если категоризировать людей по количеству детей на группы "без детей", "1 ребёнок", "2 ребёнка", "многодетные", то с этими 76 строками не всё так однозначно. Принимая, что там ошибочно дописан нолик, они попадут в категорию "2 ребёнка". Приняв же, что данные верны, они попадут в многодетную когорту. 76 строк - это 0,3%  от всех данных, не очень значительно, но всё же непосредственно относится к задаче, решаюсь заменить на медианы.
    А у 47 человек минус 1 ребёнок, допустим, случайный минус, минус надо просто убрать. 
        
    2.2. Столбец со стажем вызывает вопросики. На срезе первых или последних 20 строк встречается много отрицательных значений. Причём, если взять эти значения по модулю, не прослеживается корреляции с возрастом клиента: мы видим как маленький стаж для великовозрастных, так и могучий стаж для молодых. Всего отрицательных значений 15906 - это очень много, больше половины. Стаж должен считаться вычитанием из суммы дат увольнений суммы дат трудоустройства. Может, эти 15906 - люди с открытыми записями в трудовой, т.е. дат приёма на работу на 1 больше, чем дат увольнений, поэтому при вычитании получается минус? Есть ли тут гарантии, что устранение минуса - легальная мера? 
    Либо имел место очень некачественный перенос данных, и где-то случайно (очень много, где!) добавились чёрточки. Так как, кроме представленного датасета у нас нет никаких сырых данных, из которых мог бы быть пересчитан стаж, то ничего, кроме как взять эти числа по модулю, предпринять нельзя. Выборочные значения без минуса выглядят вполне реалистично. А также нужно пропуски заполнить средними значениями.
        
    2.3. С возрастом всё ок на первый взгляд, разве что 101 новорождённый решил взять кредит, а у некоторых даже уже есть дети. Терять 100 строк жалко, очевидно, что с возрастом ошибка, раз у кого-то проставлены дети и есть заработок. У данных по возрасту не наблюдается зашкаливающих значений, поэтому можно заменить нули на средний возраст в оставшейся когорте.
        
    2.4. Образование. Уже c первых строк наблюдается разношёрстность написания, которую надо будет привести в порядок. Разных по содержанию названий не наблюдается, так что ограничимся сведением написания в едином регистре.
        
    2.5. Семейные статусы пока выглядят нормально. Разве что для полного перфекционизма можно будет убрать прописную букву из одной категории.
        
    2.6. Существа двух полов. У нас в стране по-другому и нельзя регистрироваться. Но есть один неопределившийся, его пол звучит как "XNA". Уберу эту строку, потому что рука не поднимется присвоить какой-либо пол самостоятельно.
        
    2.7. Тип занятости, наличие задолженностей вопросов не вызвали.
        
    2.8. Ежемесячный доход у кого-то очень хороший. Отрицательных значений нет, и то хлеб. Здесь как раз лучше подойдёт заполнение пропуском медианными значениями, потому что наблюдаются довольно большие значения, они влияют на среднее.
        
    2.9. Очень рада за пенсионерку 53 лет (видимо, пенсия не по возрасту), которая собралась замуж! Если серьёзно, то скорее всего придётся объединять цели по ключевым словам, так как слишком много вариантов про одно и то же, как с жильём, например.
    
3. Имеется 54 дубликата, нужно будет подчистить.
        

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# Сначала уберём минусы из стажа и понадеемся, что ошибка только в минусе, а не в исходных расчётах. 
# Нагуглила функцию abs(), применю её и проверю, что получилось:
data['days_employed'] = data['days_employed'].abs()
data.head(15)

# Займёмся пропусками: для стажа присвоим им средние значения, для заработка - медианные. Проверим командой info():
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.info()
data.head(15)

# Посчитаем медиану столбца с детьми без учёта 20-детных строк: 
median = data.loc[data['children'] != 20, 'children'].median()
# Чтобы заработало присвоение NaN, надо загрузить ещё библиотеку:
import numpy as np
# Вычленим строки, где у нас количество детей равно 20, затем присвоим им NaN:
data.loc[data['children'] == 20, 'children'] = np.nan
# Заменим NaNы на медианные значения:
data.fillna(median,inplace=True)
# убедимся, что выпадает пустой датасет при поиске строк с 20 детьми:
data[data['children'] == 20]
# Теперь заменим на абсолютные значения столбец с количеством детей, чтобы убрать минусы перед некоторыми единицами
data['children'] = data['children'].abs()
# убедимся, что выпадает пустой датасет при поиске строк с -1 ребёнком:
data[data['children'] == -1]

# Посчитаем среднее столбца с возрастом без учёта строк с нулевым возрастом: 
mean = data.loc[data['dob_years'] != 0, 'dob_years'].mean()
# Вычленим строки, где у нас возраст равен 0, затем присвоим им NaN:
data.loc[data['dob_years'] == 0, 'dob_years'] = np.nan
# Заменим NaNы на средние значения:
data.fillna(mean,inplace=True)
# убедимся, что выпадает пустой датасет при поиске строк с нулевым возрастом:
data[data['dob_years'] == 0]

# Найдём строку с непонятным обозначением пола:
data[data['gender'] == 'XNA']
# Перенесём в новую переменную весь почищенный датасет, но уже без этой строки, уберём также старые индексы строк:
data_cleaned = data[data['gender'] != 'XNA'].reset_index(drop=True)
# Посмотрим инфо о новом датасете:
data_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null float64
days_employed       21524 non-null float64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_

### Вывод

1. NaN в колонках стажа и заработка. Отрицательный стаж.

Считаем, что данные по стажу ошибочно перенеслись с чёрточками, и уберём эти чёрточки.
Далее, если удалить строки с пропущенными данными по заработку и стажу, датасет сократится на 10%. Это довольно много. Возможно, эти данные отсутствуют, если человек всю жизнь фрилансит, не платит налоги. Может, он/она просто домохозяин/ка. Ну и само собой, может идти речь о потере данных при переносе, если эти данные собирались из разных источников, на что намекают разные написания в колонке с образованием, например. 
Раз удалять такой большой кусок жалко, и это повлияет на результат, то применим замену на усредненные (в случае стажа) и медианные (в случае заработка) значения. В колонке стажа предварительно уберём все минусы. Проверили - NaNов и отрицательных чисел в этих колонках больше нет!

2. 20-тидетные 76 человек и 47 с отрицательным ребёнком.

Допускаю, что в этих ячейках лишний нолик после двойки, но гипотетически возможно, что ошибки там и нет (очень маленькая вероятность, но есть). Заменим-ка их на медианные значения (средние дали бы дробных детей), а также заменим минус одного ребёнка на одного ребёнка. У меня получилось двухфазно: заменила эти выбивающиеся значения на NaN, а потом заменила NaN на медианы. Дети после этого утратили тип данных int64, но я позже это исправлю на шаге подобных преобразований.
Касательно отрицательных значений я допустила, что это техническая ошибка переноса, и минусы нужно просто убрать, оставив единицы.

3. 101 новорождённый.

Будем считать, что эти нулевые значения - просто потерявшиеся где-то в пути данные по возрасту, и заменим их на средний возраст по ~больнице~ когорте. Сделала это тем же способом, что и в предыдущем пункте. Тип данных надо будет вернуть в int64.

4. Существо пола "XNA".

Значения по умолчанию тут нет. Удалим эту строку. 

Итого: получили обновлённый датасет без пропусков, без аномальных количеств детей, странных возрастов и непонятного пола. Некоторые данные в нём логично считать целочисленными, поэтому следующим шагом убедимся, что они представлены в соответствующем типе данных.





### Замена типа данных

In [3]:
# Ещё раз посмотрим на инфо, чтобы ознакомиться с типами данных:
data_cleaned.info()
# Присваиваем столбцам 'children', 'days_employed', 'dob_years' тип данных int64:
data_cleaned[['children', 'days_employed', 'dob_years']] = data_cleaned[['children', 'days_employed', 'dob_years']].astype('int64')
data_cleaned.info()
data_cleaned.head(15)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null float64
days_employed       21524 non-null float64
dob_years           21524 non-null float64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_st

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Посмотрим, какие числовые данные и каких типов у нас есть:

Вещественные:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- total_income — ежемесячный доход

Целочисленные:
- education_id — идентификатор уровня образования
- family_status_id — идентификатор семейного положения
- debt — имел ли задолженность по возврату кредитов

Логично было бы перевести следующие категории в целочисленный формат:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах

Применила метод astype к этим столбцам, потому что все три нужных нам столбца уже определены как числовые, требуется только убрать знаки после запятой. В случае количества детей и возраста я исправляю побочный эффект от работы с данными из прошлого раздела, когда данные превратились в вещественные. А в случае со стажем в днях я решила сделать это преобразование, чтобы было видно, сколько целых дней отработал человек. 

После преобразований кажется, что теперь числовая часть датасета выглядит удобоваримо. Займёмся буквами..

### Обработка дубликатов

In [4]:
# уберём 54 строчки, полные аналоги которых уже имеются в датасете, убедимся, что они исчезли:
data_cleaned.drop_duplicates().reset_index(drop = True)
data_cleaned.duplicated().sum()

# Смотрим подробно, какие варианты написания образования встречаются в датасете:
data_cleaned['education'].value_counts()
# Переводим все буковки столбца 'education' в нижний регистр:
data_cleaned['education'] = data_cleaned['education'].str.lower()
# Проверяем, остались ли прописные буквы:
data_cleaned['education'].value_counts()

# Переводим все буковки столбца 'education' в нижний регистр:
data_cleaned['family_status'] = data_cleaned['family_status'].str.lower()
# Проверяем, остались ли прописные буквы:
data_cleaned['family_status'].value_counts()




женат / замужем          12380
гражданский брак          4176
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Вывод

- В начале пути мы обнаружили повторяющиеся строки, так удалим же их, облегчим немного датасет.

- Далее разберём колонку 'education'. Каждый из фактических видов образования представлен тремя способами: полностью строчными буквами, полностью прописными буквами, с большой буквы. Возможно, данные были собраны из трёх источников (отделений банка?), где сложилась своя традиция написания. Сведу все значения к нижнему регистру. Отлично, теперь у образования всего 5 честных категорий.

- Колонка с семейным положением при первичном осмотре не вызвала вопросов, кроме одной прописной буквы в категории "Не женат / не замужем". Уберу её. Можно было бы подумать над объединением категорий по смыслу, но всё же ни одна из них полностью не соответствует другой. Например, категория незамужних/неженатых должна включать в себя целиком вдовцов, разведённых, живущих в гражданском браке, но обратно это не работает: не все незамужние разведены. Поэтому я бы оставила всё как есть и при желании можно было бы объединить все "внебрачные" категории, если понадобится сравнить с официальным браком какие-то показатели.

Итого: "причесали" образование и семейное положение. Со столбцом про цель получения кредита будем разбираться далее.






### Лемматизация

In [5]:
# Посмотрим на все уникальные варианты целей кредита:
data_cleaned['purpose'].unique()

# Импортируем библиотеку с функцией лемматизации:
from pymystem3 import Mystem 
m = Mystem()

# Эта функция лемматизирует какой-то элемент строки, находящийся в столбце 'purpose':
def lemmatize_purpose(row):
    # Вытаскиваем из строки требуемый исходник, сохраняем:
    text1 = row['purpose']
    # Применяем к исходнику лемматизацию, слепляем фразу обратно, и убираем пробел+символы конца строки (rstrip)
    text2 = ' '.join(m.lemmatize(text1)).rstrip()
    # Возвращаем слепленную лемматизированную фразу:
    return text2

# Пора применить к нашему датасету функцию, и записать результаты в новый столбец:
data_cleaned['purpose_lemma'] = data_cleaned.apply(lemmatize_purpose, axis = 1)

# Смотрим, как прошла лемматизация, вытащим уникальные значения из нового столбца:
data_cleaned['purpose_lemma'].unique()
# Чукча хороший, чукча сделать лемматизация. 



array(['покупка   жилье', 'приобретение   автомобиль',
       'дополнительный   образование', 'сыграть   свадьба',
       'операция   с   жилье', 'образование', 'на   проведение   свадьба',
       'покупка   жилье   для   семья', 'покупка   недвижимость',
       'покупка   коммерческий   недвижимость',
       'покупка   жилой   недвижимость',
       'строительство   собственный   недвижимость', 'недвижимость',
       'строительство   недвижимость',
       'на   покупка   подержать   автомобиль',
       'на   покупка   свой   автомобиль',
       'операция   с   коммерческий   недвижимость',
       'строительство   жилой   недвижимость', 'жилье',
       'операция   со   свой   недвижимость', 'автомобиль',
       'заниматься   образование',
       'сделка   с   подержанный   автомобиль', 'получение   образование',
       'свадьба', 'получение   дополнительный   образование',
       'покупка   свой   жилье', 'операция   с   недвижимость',
       'получение   высокий   образование', 'свой  

### Вывод

Колонка с целями получения кредита пестрит разнообразием падежей у ключевых существительных ("жильё", например). Для последующей категоризации хорошо бы свести все эти слова к одной форме: именительный падеж, единственное число. Для этого воспользуюсь лемматизацией и создам новый столбец с полученными фразами. Так как слов в ячейке несколько, воспользуюсь join, чтобы слепить их обратно во "фразы".
Получилось! Теперь можно найти всё, что нужно, по ключевым словам, и отнести к выбранным категориям.


### Категоризация данных

In [6]:
# Новая функция! Она возьмет фразу из лемматизированной колонки с целями кредита, поищет в ней ключевые слова
# и вернёт одну из четырёх категорий, к которой эта фраза относится: недвижимость, автомобили, свадьба или образование:
def categorize_purpose(row):
    text = row['purpose_lemma']
    if 'жилье' in text or 'недвижимость' in text:
        return 'недвижимость'
    elif 'автомобиль' in text:
        return 'автомобили'
    elif 'образование' in text:
        return 'образование'
    elif 'свадьба' in text:
        return 'свадьба'
    else:
        return 'хз'
    
# Применим функцию категоризации цели кредита ко всему столбцу лемматизированных целей в нашем датасете:
data_cleaned['purpose_category'] = data_cleaned.apply(categorize_purpose, axis = 1)
# Проверим, не получилось ли категорий больше четырёх задуманных:
data_cleaned['purpose_category'].unique()

# Эта функция разбивает все имеющиеся в датасете семейные положения на два типа: замужние/незамужние
# и женатые/неженатые.
def categorize_family_status(row):
    text = row['family_status']
    if text == 'женат / замужем':
        return 'женат/замужем'
    else:
        return 'не женат/не замужем'
    
# Применим функцию категоризации семейного положения ко всему столбцу семейных положений в нашем датасете:
data_cleaned['family_status_category'] = data_cleaned.apply(categorize_family_status, axis = 1)
# Проверим, как сработала функция, и заодно узнаем, сколько женатиков:
data_cleaned['family_status_category'].value_counts()

# Функция категоризирует количество детей указанных в датасете граждан на 4 группы: без детей, один ребёнок,
# двое детей, многодетный:
def categorize_children(row):
    child = row['children']
    if child == 0:
        return 'без детей'
    elif child == 1:
        return 'один ребёнок'
    elif child == 2:
        return 'двое детей'
    else:
        return 'многодетный'
    
# Применим функцию категоризации количества детей ко всему столбцу детишек в нашем датасете:
data_cleaned['children_category'] = data_cleaned.apply(categorize_children, axis = 1)
# Проверим, как сработала функция, и заодно посмотрим, как с демографией у этого среза общества:
data_cleaned['children_category'].value_counts()

# Функция категоризирует доход указанных в датасете граждан на группы (в тыс.руб.): 0-50, 50-100, 100-150, 150-200,
# 200-300, 300-500, 500+:
def categorize_income(row):
    income = row['total_income']
    if income <= 50000:
        return '0-50k'
    elif 50001 <= income < 100000:
        return '50-100k'
    elif 100001 <= income < 150000:
        return '100-150k'
    elif 150001 <= income < 200000:
        return '150-200k'
    elif 200001 <= income < 300000:
        return '200-300k'
    elif 300001 <= income < 500000:
        return '300-500k'
    else:
        return '500k+'
    
# Применим функцию категоризации дохода ко всему столбцу доходов в нашем датасете:
data_cleaned['total_income_category'] = data_cleaned.apply(categorize_income, axis = 1)
# Проверим, как сработала функция, и заодно посмотрим, как распределены доходы:
data_cleaned['total_income_category'].value_counts()


# data_cleaned.head()

100-150k    7878
150-200k    4118
50-100k     4091
200-300k    3583
300-500k    1260
0-50k        372
500k+        222
Name: total_income_category, dtype: int64

### Вывод

По существу среди целей получения кредита, слегка обобщая (например, новые и старые автомобили, или коммерческую и частную недвижимость), получаем всего-то 4 категории: недвижимость(жильё), автомобили, свадьба, образование. Создаю ещё одну колонку, там проставлю одну из этих четырёх категорий. Для этого создам функцию, которая будет подставлять нужную категорию, и применю её построчно в датасете. Функция получилась небыстрая, со временем надеюсь освоить более хитрые способы.

Ещё очень хочется категоризировать все семейные положения на два типа: "женат/замужем" и "не женат/не замужем". Так можно посмотреть более упрощённо на влияние именно зарегистрированного брака на возврат кредита, игнорируя довольно субъективные категории. Их мы тоже посмотрим, хотя, к сожалению, из них всё равно сложно понять, одинокий человек, или у него общий с кем-то бюджет. Категоризировали! Женатых несколько больше, чем неженатых. Возможно, у последних меньше поводов брать кредиты).

Интересно было бы категоризировать количество детей, потому что один ребёнок обходится значительно дешевле двух, хоть и не в два раза. Многодетные же тратят ещё больше, правда, у них больше всяких льгот, может быть снижена потребность в кредитах. В общем, я бы изучила этот вопрос, тем более, в задаче упоминалось о количестве детей. 

Ещё я бы категоризировала возраст, но про него в задаче не спрашивается, оставлю его в покое.

Осталось решить по поводу доходов. Чтобы ответить на вопрос, как уровень доходов влияет на возврат кредита, надо понять, что мы называем уровнем дохода: точное число или действительный уровень вроде "низкий"/"средний"/"заоблачный" и тд. Второе, скорее всего, в нашей системе координат изучить не получится, потому что неизвестна география этих данных, а богатый человек в Кандалакше может и не сойти за богатого в Петербурге. Голые цифры в количестве 20к всё равно нужно как-то категоризировать, нужны же нам группы сравнения, сделаю бездушное разделение на группы, увеличив разницу после 200к: 0-50к, 50-100к, 100-150к, 150-200к, 200-300к, 300-500к, 500к и выше. Опять пишу функцию, опять применяю построчно, на сей раз для столбца с доходом.

Вопросов к таким данным можно задать великое множество, но мне нужно ответить только на несколько из них. Приступим к анализу.


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# Обрезаем наш датасет до двух колонок: категория детности и наличие задолженности.
# Наличие задолженности обозначается единицей, отсутствие - нулём.
children_debt = data_cleaned[['children_category', 'debt']]

# Слепляем два датафрейма, сгруппированных по категории детности: общее количество заёмщиков и
# процент заёмщиков, верувших кредит. 
# Процент в каждой группе считается: (количество строк - сумма единиц)*100 / количество строк.
# Там же округляем проценты до 2х знаков после запятой.
children_result = pd.merge((children_debt.groupby('children_category').count()), 
(((children_debt.groupby('children_category').count() - children_debt.groupby('children_category').sum()) * 100 / 
children_debt.groupby('children_category').count()).round(2)), how = 'left', 
on = 'children_category')

# Добавим названия колонкам для пояснения, где у нас что:
children_result.columns = ['количество заёмщиков', '% вернувших кредит']
# Отсортируем результирующую таблицу по убыванию процента "возвращателей" кредита:
children_result.sort_values(by = '% вернувших кредит', ascending = False)



,количество заёмщиков,% вернувших кредит
children_category,,
без детей,14224,92.47
многодетный,380,91.84
один ребёнок,4865,90.85
двое детей,2055,90.56


### Вывод

- Возврат кредита среди бездетных несколько выше, чем среди детных, но в целом, среди всех категорий 90% вернули долги. 
- Интересно, что многодетные чуть более сознательны, чем граждане с одним или двумя детьми, возможно, из-за наличия каких-то дополнительных льгот/выплат от государства, которые позволяют более стабильно поддерживать семейный бюджет. Или многодетность делает людей более взвешенными и осторожными, и они берут кредиты, точно убедившись, что смогут  их выплачивать.
- Однодетные и двудетные примерно одинаково платёжеспособны, что опровергло гипотезу, что появление второго ребёнка сильно бьёт по бюджету.

- Важно в конце отметить, что сами группы представлены очень неравномерно: даже есть суммировать всех заёмщиков с детьми, они всё равно не "догонят" бездетных. Тем не менее, все (кроме многодетных) выборки сравнительно большие, можно доверять полученным процентам.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# Обрезаем наш датасет до двух колонок: семейное положение (бинарное) и наличие задолженности.
# Наличие задолженности обозначается единицей, отсутствие - нулём.
family_status_debt = data_cleaned[['family_status_category', 'debt']]

# Слепляем два датафрейма, сгруппированных по категории семейного положения (бинарного): общее количество заёмщиков и
# процент заёмщиков, верувших кредит. 
# Процент в каждой группе считается: (количество строк - сумма единиц)*100 / количество строк.
# Там же округляем проценты до 2х знаков после запятой.
family_status_result = pd.merge((family_status_debt.groupby('family_status_category').count()), 
(((family_status_debt.groupby('family_status_category').count() - family_status_debt.groupby('family_status_category').sum()) * 100 / 
family_status_debt.groupby('family_status_category').count()).round(2)), how = 'left', 
on = 'family_status_category')

# Добавим названия колонкам для пояснения, где у нас что:
family_status_result.columns = ['количество заёмщиков', '% вернувших кредит']
# Отсортируем результирующую таблицу по убыванию процента "возвращателей" кредита:
print(family_status_result.sort_values(by = '% вернувших кредит', ascending = False))


# Обрезаем наш датасет до двух колонок: семейное положение (5 категорий) и наличие задолженности.
# Наличие задолженности обозначается единицей, отсутствие - нулём.
family_status2_debt = data_cleaned[['family_status', 'debt']]

# Слепляем два датафрейма, сгруппированных по категории семейного положения: общее количество заёмщиков и
# процент заёмщиков, верувших кредит. 
# Процент в каждой группе считается: (количество строк - сумма единиц)*100 / количество строк.
# Там же округляем проценты до 2х знаков после запятой.
family_status2_result = pd.merge((family_status2_debt.groupby('family_status').count()), 
(((family_status2_debt.groupby('family_status').count() - family_status2_debt.groupby('family_status').sum()) * 100 / 
family_status2_debt.groupby('family_status').count()).round(2)), how = 'left', 
on = 'family_status')

# Добавим названия колонкам для пояснения, где у нас что:
family_status2_result.columns = ['количество заёмщиков', '% вернувших кредит']
# Отсортируем результирующую таблицу по убыванию процента "возвращателей" кредита:
family_status2_result.sort_values(by = '% вернувших кредит', ascending = False)



                        количество заёмщиков  % вернувших кредит
family_status_category                                          
женат/замужем                          12380               92.48
не женат/не замужем                     9144               91.14


,количество заёмщиков,% вернувших кредит
family_status,,
вдовец / вдова,960,93.44
в разводе,1195,92.89
женат / замужем,12380,92.48
гражданский брак,4176,90.71
не женат / не замужем,2813,90.26


### Вывод

- Женатые/замужние возвращают кредиты несколько лучше незамужних/неженатых.

- Посмотрим подробнее на неженатых/незамужних. Интересно: овдовевшие и разведённые платёжеспособны даже больше чем женатые/замужние! Наверное, потому, что такие категории предполагают, что человек всё же в данный момент живёт один, и не обременён необходимостью финансовой поддержки близкого человека.

- Самые "ненадёжные" категории - незамужние/неженатые и живущие в гражданском браке. Возможно, потому что груз ответственности у них ниже, чем у людей со штампом в паспорте, и это каким-то образом проецируется на отношение к возврату долгов.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
# Обрезаем наш датасет до двух колонок: уровень дохода (7 категорий) и наличие задолженности.
# Наличие задолженности обозначается единицей, отсутствие - нулём.
total_income_category_debt = data_cleaned[['total_income_category', 'debt']]

# Слепляем два датафрейма, сгруппированных по категории дохода: общее количество заёмщиков и
# процент заёмщиков, верувших кредит. 
# Процент в каждой группе считается: (количество строк - сумма единиц)*100 / количество строк.
# Там же округляем проценты до 2х знаков после запятой.
total_income_category_result = pd.merge((total_income_category_debt.groupby('total_income_category').count()), 
(((total_income_category_debt.groupby('total_income_category').count() - 
   total_income_category_debt.groupby('total_income_category').sum()) * 100 / 
total_income_category_debt.groupby('total_income_category').count()).round(2)), how = 'left', 
on = 'total_income_category')

# Добавим названия колонкам для пояснения, где у нас что:
total_income_category_result.columns = ['количество заёмщиков', '% вернувших кредит']
# Отсортируем результирующую таблицу по убыванию процента "возвращателей" кредита:
total_income_category_result.sort_values(by = '% вернувших кредит', ascending = False)

,количество заёмщиков,% вернувших кредит
total_income_category,,
0-50k,372,93.82
500k+,222,93.69
200-300k,3583,92.97
300-500k,1260,92.70
50-100k,4091,91.91
100-150k,7878,91.61
150-200k,4118,91.06


### Вывод

- Линейной зависимости не наблюдается: платёжеспособность не растёт с увеличением дохода.
- Самые ответственные - граждане с доходом до 50 тыс. рублей. Они считают каждую копейку, это позволяет учесть риски перед тем, как брать в долг.
- Платёжеспособность людей с доходами выше 200 тыс. рублей ожидаемо выше, чем у зарабатывающих менее 200 тыс. руб. Большинству из них хватает и на комфортную жизнь, и на возврат кредита. 
- "Средняя" категория самая нестабильная. Оно и понятно, средний класс всегда более реактивен к экономическим изменениям. 
- Самые неплатёжеспособные - 150-200к - но современные меркам это некая переходная стадия: люди уже близки к "лучшей жизни", но немного до неё не дотягивают, приходится принимать много финансовых решений в пользу того или другого. Такая обстановка не располагает к стабильной платёжеспособности.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
# Обрезаем наш датасет до двух колонок: уровень дохода (4 категории) и наличие задолженности.
# Наличие задолженности обозначается единицей, отсутствие - нулём.
purpose_category_debt = data_cleaned[['purpose_category', 'debt']]

# Слепляем два датафрейма, сгруппированных по категории дохода: общее количество заёмщиков и
# процент заёмщиков, верувших кредит. 
# Процент в каждой группе считается: (количество строк - сумма единиц)*100 / количество строк.
# Там же округляем проценты до 2х знаков после запятой.
purpose_category_result = pd.merge((purpose_category_debt.groupby('purpose_category').count()), 
(((purpose_category_debt.groupby('purpose_category').count() - 
   purpose_category_debt.groupby('purpose_category').sum()) * 100 / 
purpose_category_debt.groupby('purpose_category').count()).round(2)), how = 'left', 
on = 'purpose_category')

# Добавим названия колонкам для пояснения, где у нас что:
purpose_category_result.columns = ['количество заёмщиков', '% вернувших кредит']
# Отсортируем результирующую таблицу по убыванию процента "возвращателей" кредита:
purpose_category_result.sort_values(by = '% вернувших кредит', ascending = False)

,количество заёмщиков,% вернувших кредит
purpose_category,,
недвижимость,10839,92.79
свадьба,2348,92.08
образование,4022,90.80
автомобили,4315,90.66


### Вывод

- Категория "недвижимость" представлена больше всех в нашем датасете и побеждает в рейтинге платежеспособности. Вероятно, это вызвано тем, что сами по себе сделки с недвижимостью на порядок серьёзнее остальных категорий. Это добавляет основательности к подготовке всех сопутствующих процессов. Клиент долго взвешивает все за и против, суммы подобных кредитов очень высоки, надо всё хорошо спрогнозировать. Вторая возможная причина: цены на недвижимость сейчас таковы, что даже люди с приличным достатком могут не осилить купить квартиру только на свои накопления, ипотеку в большинстве случаев позволяют себе люди с хорошими финансовыми "подушками" и стабильным заработком. Отсюда и высокий процент возвратов.
- Свадебные кредиты на втором месте, скорее всего, из-за краткосрочности. Этот кредит чаще всего возвращать не очень долго, и будущие молодожёны в состоянии спрогнозировать, смогут ли, запример, за год вернуть все деньги. Да и родители помогут!!
- Образование. Предполагаю, что платёжеспособность тут ниже от того, что у части "образовавшихся" результат не оправдывает ожиданий. Либо они находят работу по специальности, но начинать приходится с малого дохода, и какой уж тут возврат кредита. Либо они сделали неправильный выбор профессии в самом начале, не справились, и так и остались на старом месте и низкой зарплате.
- За автомобили долги возвращают хуже всего. В голову приходит только импульсивный характер подобных покупок (как с айфонами), желание стать обладателем новой крутой тачки сильнее последствий. Остальные же 90,66% большие молодцы!

### Шаг 4. Общий вывод

Что ж, в процессе анализа датасета выявились как ожидаемые закономерности, так и интересные находки.
Если суммировать лучшие каагории заёмщиков из всех сравнений, получаем:

Лучше всего выдавать кредит на жильё семейному или очевидно одинокому человеку без детей (либо многодетному), доход у него должен быть либо очень низким (меньше 50к), либо выше 200к.
Странно, что свадебная категория тоже работает неплохо в плане возврата долгов, ведь назамужние/неженатые и живущие в гражданском браке самые ненадёжные. Интересно было бы сравнить одновременно категории семейного положения и цели.


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.